# MovieMatcher — Notebook Colab

**Objetivo:** entrenar un modelo compacto (distilgpt2) con el dataset *Wikipedia Movie Plots* (Kaggle) para generar recomendaciones y tramas similares a partir de una descripción del usuario.

**Instrucciones rápidas:**

1. Sube tu `kaggle.json` (API token) usando el icono de archivos en Colab o ejecuta la celda de upload.

2. Ejecuta las celdas en orden. Ajusta `N_EXAMPLES` y `NUM_EPOCHS` según recursos de Colab.

> Nota: el entrenamiento puede consumir recursos. Para demo rápido, usa `N_EXAMPLES = 500` y `NUM_EPOCHS = 1`.

In [ ]:
## 1) Instalar dependencias (ejecutar)
# Si ya tienes torch instalado con GPU, puedes ajustar la instalación de torch apropiada.
!pip install -q transformers datasets gradio kaggle pandas sentencepiece torch


In [ ]:
## 2) Subir kaggle.json (si no lo has subido manualmente)
from google.colab import files
print('Si ya subiste ~/.kaggle/kaggle.json omite este paso.')
uploaded = files.upload()
print('Archivos subidos:', list(uploaded.keys()))


In [ ]:
## 3) Configurar Kaggle (si subiste kaggle.json)
import os
if os.path.exists('kaggle.json'):
    os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
    !cp kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    print('kaggle.json copiado a ~/.kaggle/kaggle.json') 
else:
    print('No se encontró kaggle.json en el directorio actual. Si lo subiste con el diálogo, recarga el kernel.')


In [ ]:
## 4) Descargar dataset Wikipedia Movie Plots desde Kaggle
# Dataset: jrobischon/wikipedia-movie-plots
!kaggle datasets download -d jrobischon/wikipedia-movie-plots -p /content --unzip -q
import os
print('Archivos en /content:', os.listdir('/content')[:20])


In [ ]:
## 5) Cargar y explorar el dataset con pandas
import pandas as pd
import os
csv_path = '/content/wiki_movie_plots_deduped.csv'
if not os.path.exists(csv_path):
    files = os.listdir('/content')
    candidates = [f for f in files if f.endswith('.csv')]
    if len(candidates) > 0:
        csv_path = os.path.join('/content', candidates[0])
    else:
        raise FileNotFoundError('No se encontró el CSV del dataset. Asegúrate de que la descarga/zip funcionó.')
print('Usando:', csv_path)
df = pd.read_csv(csv_path)
print('Tamaño del dataset:', len(df))
df = df.dropna(subset=['Plot', 'Title'])
df = df.reset_index(drop=True)
df.head(3)


In [ ]:
## 6) Preprocesamiento básico
import re
def limpiar_texto(t):
    t = str(t)
    t = re.sub(r'\\s+', ' ', t)
    t = re.sub(r'\[[^\]]*\]', '', t)
    return t.strip()

pd.options.display.max_colwidth = 200
df['Plot_clean'] = df['Plot'].apply(limpiar_texto)
df['plot_len'] = df['Plot_clean'].str.split().apply(len)
print('longitud media (palabras):', int(df['plot_len'].mean()))
df = df[(df['plot_len'] >= 20) & (df['plot_len'] <= 800)].reset_index(drop=True)
print('Después de filtrar por longitud:', len(df))
df[['Title','Plot_clean']].sample(3, random_state=42)


In [ ]:
## 7) Construir pares Prompt -> Recomendación (titles) y Prompt -> Sinopsis (opcional)
N_EXAMPLES = 2000  # Ajusta para Colab gratuito: prueba con 500 o 1000
df_small = df.sample(n=min(N_EXAMPLES, len(df)), random_state=42).reset_index(drop=True)

pairs = []
for idx, row in df_small.iterrows():
    words = row['Plot_clean'].split()
    prompt = ' '.join(words[:35])
    title = row['Title']
    short_plot = ' '.join(words[:70])
    pairs.append({'prompt': prompt, 'target_title': title, 'target_plot': short_plot})

import pandas as pd
pairs_df = pd.DataFrame(pairs)
print('Ejemplos de pares:')
pairs_df.head(5)


In [ ]:
## 8) Construir dataset de entrenamiento en formato Hugging Face
from datasets import Dataset
MODE = 'title'  # 'title' o 'plot'

if MODE == 'title':
    hf_examples = [{'text': f'Prompt: {r["prompt"]}  Recomendacion:' + ' ' + r['target_title']} for _, r in pairs_df.iterrows()]
else:
    hf_examples = [{'text': f'Prompt: {r["prompt"]}  Sinopsis:' + ' ' + r['target_plot']} for _, r in pairs_df.iterrows()]

ds = Dataset.from_list(hf_examples)
print('Dataset HF creado. Ejemplos:')
ds.select(range(3))


In [ ]:
## 9) Tokenizador y modelo (distilgpt2) — preparar pad token y resize embeddings
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token
model.config.pad_token_id = model.config.get('pad_token_id', tokenizer.convert_tokens_to_ids(tokenizer.pad_token))

print('Vocab size:', len(tokenizer))
print('Modelo y tokenizer listos.')


In [ ]:
## 10) Tokenizar y crear labels (labels = input_ids) para causal LM
MAX_LENGTH = 128

def tokenize_and_label(batch):
    toks = tokenizer(batch['text'], truncation=True, padding='max_length', max_length=MAX_LENGTH)
    toks['labels'] = toks['input_ids'].copy()
    return toks

print('Tokenizando dataset... (esto puede tardar unos minutos según N_EXAMPLES)')
ds_tok = ds.map(tokenize_and_label, batched=True, remove_columns=['text'])
print(ds_tok)


In [ ]:
## 11) Preparar Trainer y TrainingArguments
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

NUM_EPOCHS = 1  # aumentar si tienes recursos
BATCH_SIZE = 2

training_args = TrainingArguments(
    output_dir='./movie_matcher_model',
    overwrite_output_dir=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tok,
    data_collator=data_collator
)

print('Trainer listo. Para entrenar ejecuta trainer.train()')


In [ ]:
## 12) Entrenar el modelo (descomenta para ejecutar)
# Atención: en Colab gratuito el entrenamiento puede tardar y consumir memoria.
# Si quieres probar rápido, reduce N_EXAMPLES y NUM_EPOCHS arriba.
# trainer.train()
# trainer.save_model('./movie_matcher_model')
print('Si quieres entrenar, descomenta trainer.train() y trainer.save_model(...) en esta celda.')


In [ ]:
## 13) Función para generar recomendaciones (usa el modelo cargado en memoria)
import torch

def generar_recomendacion(prompt, max_length=80, temperature=1.0, top_p=0.92, top_k=50, repetition_penalty=1.8, no_repeat_ngram_size=3):
    model.eval()
    with torch.no_grad():
        entrada = f'Prompt: {prompt}  Recomendacion:'
        inputs = tokenizer(entrada, return_tensors='pt')
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=True,
            num_return_sequences=1
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.replace(entrada, '').strip()


In [ ]:
## 14) Interfaz Gradio para probar (ejecuta esta celda después de entrenar o si cargas un modelo guardado)
import gradio as gr

def gradio_fn(prompt, max_length, temp, top_p, top_k, rep_penalty, no_repeat):
    return generar_recomendacion(prompt, max_length=int(max_length), temperature=float(temp), top_p=float(top_p), top_k=int(top_k), repetition_penalty=float(rep_penalty), no_repeat_ngram_size=int(no_repeat))

iface = gr.Interface(
    fn=gradio_fn,
    inputs=[
        gr.Textbox(lines=2, label='Describe lo que quieres ver'),
        gr.Slider(20, 200, value=80, label='max_length'),
        gr.Slider(0.1, 2.0, value=1.0, label='temperature'),
        gr.Slider(0.1, 1.0, value=0.92, label='top_p'),
        gr.Slider(1, 200, value=50, step=1, label='top_k'),
        gr.Slider(1.0, 3.0, value=1.8, step=0.1, label='repetition_penalty'),
        gr.Slider(1, 5, value=3, step=1, label='no_repeat_ngram_size'),
    ],
    outputs=gr.Textbox(label='Recomendación / Texto generado'),
    title='MovieMatcher - Recomendador de películas (demo)',
    description='Introduce una breve descripción y el modelo sugerirá títulos o tramas (según el modo).'
)

# Lanza la interfaz (en Colab mostrará un enlace público)
iface.launch(enable_queue=True, share=True)


---

### Notas finales

- Ajusta `MODE`, `N_EXAMPLES`, `NUM_EPOCHS`, `MAX_LENGTH` y `BATCH_SIZE` según los recursos de Colab.
- Para una demo rápida: `MODE='title'`, `N_EXAMPLES=500`, `NUM_EPOCHS=1`, `BATCH_SIZE=2`.
- Si vas a publicar el modelo, recuerda citar el dataset de Kaggle y respetar licencias.
